In [10]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

with open('mapbox_token', 'r') as f:
    token = f.read()
    
init_notebook_mode(connected=True)

In [25]:
import pandas
test_data = pandas.read_csv('testdata.csv')

In [34]:
import numpy
plot_data = {
    'lat': test_data.lat[:200],
    'lon': test_data.lon[:200],
    'time': test_data.time[:200]
}
poi = {
    'lat': numpy.average(plot_data['lat']),
    'lon': numpy.average(plot_data['lon'])
}

In [37]:
def animate_movement(plot_data, poi, zoom_level):
    """plot_data is a dictionary with keys latitude, longitude,
    and timestamp. Each is a list of the same length.
    poi is the coordinate of the center of the map. dict with keys lat and lon.
    zoom_level should be between 1 - 15.
    returns figure
    """
    # create initial data
    initial_data = go.Scattermapbox(
        lat = plot_data['lat'][0],
        lon = plot_data['lon'][0],
        mode = 'markers',
        marker = {'size': 4},
        text = [plot_data['time'][0]],
    )
    # create layout
    layout = go.Layout(
        autosize = True,
        hovermode = 'closest',
        mapbox = {
            'accesstoken': token,
            'bearing': 0,
            'center': poi,
            'pitch': 0,
            'zoom': zoom_level
        },
        sliders = []
    )
    # generate frames and steps in sliders
    frames = []
    steps = []
    for i in range(len(plot_data['time'])):
        data = go.Scattermapbox(
            lat = plot_data['lat'][:i],
            lon = plot_data['lon'][:i],
            mode = 'markers',
            marker = {'size': 4},
            # hover mode seem not to work in the notebook
            text = plot_data['time'][:i]
        )
        frames.append({'data': [data], 'name': plot_data['time'][i]})
        steps.append({
            'args': [
                [plot_data['time'][i]],
                {'frame': {'duration': 200, 'redraw': False},
                 'mode': 'immediate',
                 'transition': {'duration': 200}}
            ],
            'method': 'animate'}
        )
    # put together the figure
    layout['sliders'] = [{'steps': steps}]
    figure = {
        'data': [initial_data],
        'layout': layout,
        'frames': frames
    }
    return figure

In [38]:
figure = animate_movement(plot_data, poi, 12)
iplot(figure)